In [52]:
import random
import itertools
from concurrent.futures import ThreadPoolExecutor, as_completed

import requests
import pandas as pd
import speaker_buddy as buddy

### Exchanges

In [16]:
EXCHANGES= [
    'bitfinex',
    'coinbase-pro',
    'bitstamp',
    'kraken',
    'cexio',
    'okcoin',
    'bitmex',
    'mexbt',
    'huobi',
    'poloniex',
    'bittrex',
    'okex',
    'hitbtc'
]

### Symbols

In [17]:
SYMBOLS = ['btc', 'ltc', 'eth']

### Combinations

In [35]:
ALL_COMBINATIONS = list(itertools.product(EXCHANGES, SYMBOLS))

### Random Combinatios

In [36]:
combinations = [random.choice(ALL_COMBINATIONS) for _ in range(10)]

### Random Dates

In [29]:
ALL_DATES = [d.strftime('%Y-%m-%d') for d in pd.date_range('2019-01-01', '2019-07-20', freq='D')]

In [30]:
dates = [random.choice(ALL_DATES) for _ in range(5)]

### Summary

In [31]:
dates

['2019-07-04', '2019-02-10', '2019-03-08', '2019-04-24', '2019-07-02']

In [37]:
combinations

[('cexio', 'ltc'),
 ('okex', 'ltc'),
 ('poloniex', 'btc'),
 ('coinbase-pro', 'eth'),
 ('cexio', 'btc'),
 ('hitbtc', 'ltc'),
 ('okcoin', 'btc'),
 ('hitbtc', 'btc'),
 ('okcoin', 'btc'),
 ('kraken', 'btc')]

In [40]:
data = list(itertools.product(combinations, dates))
data[:3]

[(('cexio', 'ltc'), '2019-07-04'),
 (('cexio', 'ltc'), '2019-02-10'),
 (('cexio', 'ltc'), '2019-03-08')]

In [38]:
URL = 'http://localhost:5000/price/{exchange}/{symbol}/{date}'

In [48]:
price_urls = [
        URL.format(exchange=exchange, symbol=symbol, date=date)
    for (exchange, symbol), date in data]

In [49]:
price_urls[:3]

['http://localhost:5000/price/cexio/ltc/2019-07-04',
 'http://localhost:5000/price/cexio/ltc/2019-02-10',
 'http://localhost:5000/price/cexio/ltc/2019-03-08']

---

# Single Threaded

In [38]:
URL = 'http://localhost:5000/price/{exchange}/{symbol}/{date}'

In [45]:
with buddy.measure_time() as tm:
    responses = [requests.get(url) for url in price_urls]

print([r.json() for r in responses])

[{'exchange': 'cexio', 'symbol': 'ltc', 'open': 119.29, 'high': 123.41, 'low': 118, 'close': 121.83, 'volume': 520.39092011, 'day': '2019-07-04'}, {'exchange': 'cexio', 'symbol': 'ltc', 'open': 43.68, 'high': 47.85, 'low': 42.81, 'close': 44.67, 'volume': 1265.54319828, 'day': '2019-02-10'}, {'exchange': 'cexio', 'symbol': 'ltc', 'open': 56, 'high': 59.86, 'low': 54.65, 'close': 57.96, 'volume': 1115.05720511, 'day': '2019-03-08'}, {'exchange': 'cexio', 'symbol': 'ltc', 'open': 77.26, 'high': 80, 'low': 74.86, 'close': 74.86, 'volume': 434.31614652, 'day': '2019-04-24'}, {'exchange': 'cexio', 'symbol': 'ltc', 'open': 122.4, 'high': 126.81, 'low': 113.16, 'close': 123.65, 'volume': 993.20811934, 'day': '2019-07-02'}, {'exchange': 'okex', 'symbol': 'ltc', 'open': 118.75, 'high': 123.15, 'low': 116.8, 'close': 121.32, 'volume': 606568.249829, 'day': '2019-07-04'}, None, {'exchange': 'okex', 'symbol': 'ltc', 'open': 55.3752, 'high': 59.2, 'low': 53.9053, 'close': 56.7172, 'volume': 2452158

In [46]:
tm.print()

Total duration: 10.85 seconds


# Multi threaded

In [53]:
def get_price(url):
    resp = requests.get(url)
    return resp.json()

In [54]:
with buddy.measure_time() as tm:
    with ThreadPoolExecutor() as ex:
        futures = [ex.submit(get_price, url) for url in urls]

    for future in as_completed(futures):
        print(future.result())

{'exchange': 'cexio', 'symbol': 'ltc', 'open': 43.68, 'high': 47.85, 'low': 42.81, 'close': 44.67, 'volume': 1265.54319828, 'day': '2019-02-10'}
{'exchange': 'cexio', 'symbol': 'ltc', 'open': 119.29, 'high': 123.41, 'low': 118, 'close': 121.83, 'volume': 520.39092011, 'day': '2019-07-04'}
{'exchange': 'okex', 'symbol': 'ltc', 'open': 118.75, 'high': 123.15, 'low': 116.8, 'close': 121.32, 'volume': 606568.249829, 'day': '2019-07-04'}
{'exchange': 'cexio', 'symbol': 'ltc', 'open': 56, 'high': 59.86, 'low': 54.65, 'close': 57.96, 'volume': 1115.05720511, 'day': '2019-03-08'}
{'exchange': 'cexio', 'symbol': 'ltc', 'open': 77.26, 'high': 80, 'low': 74.86, 'close': 74.86, 'volume': 434.31614652, 'day': '2019-04-24'}
None
{'exchange': 'cexio', 'symbol': 'ltc', 'open': 122.4, 'high': 126.81, 'low': 113.16, 'close': 123.65, 'volume': 993.20811934, 'day': '2019-07-02'}
{'exchange': 'okex', 'symbol': 'ltc', 'open': 76.47, 'high': 79.31, 'low': 73.28, 'close': 73.68, 'volume': 1044648.7787879999, 

In [55]:
tm.print()

Total duration: 0.91 seconds
